# Preliminary Data

### Setup

In [1]:
from datetime import datetime

print("\033[32m{}\033[0m".format(datetime.now().strftime("%B %d, %Y %H:%M:%S")))

March 07, 2022 10:22:46


In [2]:
import pandas as pd
from pandas import DataFrame
from utils import MostFrequentlyDisruptedPartnersAdder

INTERACTIONS_SUMMARY_TABLE_FOLDER_PATH = r"../dev/interactions_summary_tables"

# Preliminary Data Paths
BRCA_PRELIMINARY_DATA_PATH = r"../../data/predictions_datasets/brca_prediction_2021-11-17/558297b6/brca_preliminary_data_cgc_2021-11-17.xlsx"
COAD_PRELIMINARY_DATA_PATH = r"../../data/predictions_datasets/coad_prediction_2022-01-06/9789393f/coad_preliminary_data_cgc_2022-02-16.xlsx"
ESCA_PRELIMINARY_DATA_PATH = r"../../data/predictions_datasets/esca_prediction_2021-11-17/59544312/esca_preliminary_data_cgc_2021-11-17.xlsx"
GBM_PRELIMINARY_DATA_PATH = r"../../data/predictions_datasets/gbm_prediction_2021-11-17/b9cfc7ea/gbm_preliminary_data_cgc_2021-11-17.xlsx"
HNSC_PRELIMINARY_DATA_PATH = r"../../data/predictions_datasets/hnsc_prediction_2021-11-17/53930f30/hnsc_preliminary_data_cgc_2021-11-17.xlsx"
OV_PRELIMINARY_DATA_PATH = r"../../data/predictions_datasets/ov_prediction_2021-11-17/37bf1637/ov_preliminary_data_cgc_2021-11-17.xlsx"

# Reflect changes in the modules immediately.
%load_ext autoreload
%autoreload 2

### Filtering Condition

### Updated:
[14:29, 03/03/2022] Hilal Kazan (ABU CS Prof. ): bu tabloda disruptive ratio 0.8 ve üstü desek daha iyi olacak, o zaman sığar sanıyorum
[14:29, 03/03/2022] Hilal Kazan (ABU CS Prof. ): COADS Q13748 için TUBA3C gen id'si

[15:51, 03/03/2022] Hilal Kazan (ABU CS Prof. ): Tablo 2 ile ilgili birkaç değişiklik
[15:51, 03/03/2022] Hilal Kazan (ABU CS Prof. ): COAD'ı BRCA'dan sonraya taşıyalım. Bir de aynı kohort içindeki genleri interface count'a göre sort edelim.
[15:51, 03/03/2022] Hilal Kazan (ABU CS Prof. ): Bir de aynı kohort içindeki genleri interface count'a göre sort edelim. yerine şu daha doğru önce disruptive ratio sonra interface count'a göre.
[15:52, 03/03/2022] Hilal Kazan (ABU CS Prof. ): Örneğin COAD'da 13 gen 1 disruptive _ratio'ya sahip,
[15:52, 03/03/2022] Hilal Kazan (ABU CS Prof. ): bu genler rastgele sıralanmasın, interface count'ı en yüksek olan en yukarıda olsun


[14:54, 04/03/2022] Hilal Kazan (ABU CS Prof. ): patient count'u 1 ve 2 olanları çıkartmak lazım bence
[14:54, 04/03/2022] Hilal Kazan (ABU CS Prof. ): bir de nan'ları düzeltelim

In [3]:
def round_decimals(x, n_decimals=2):
    if isinstance(x, float):
        try:
            return round(x, n_decimals)
        except TypeError:
            print(f"VALUE X={x}")
    else:
        return x

In [4]:
def filterer(data, tcga) -> DataFrame:
    data_filtered = data.copy()
    # Disruptive ratio
    data_filtered["RATIO_(prev_two_col)"] = data_filtered["NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR"] / data_filtered["PATIENT_INTERFACE_COUNT"]
    # Round the ratio values.
    data_filtered["RATIO_(prev_two_col)"] = data_filtered["RATIO_(prev_two_col)"].apply(lambda x: round_decimals(x))
    # Filtering based on given conditions.
    data_filtered = data_filtered[
        (data_filtered["PATIENT_INTERFACE_COUNT"] >= 5) &
        (data_filtered["RATIO_(prev_two_col)"] >= 0.8)
    ].copy()

    # Keep only relevant columns
    data_filtered = data_filtered[
        ["PROTEIN", "GENE", "PATIENT_CORE_COUNT", "PATIENT_INTERFACE_COUNT", "NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR", "RATIO_(prev_two_col)", "NUM_UNIQUE_INTERACTORS", "CGC_STATUS"]
    ].copy()

    data_filtered.insert(0, "TCGA", tcga)

    data_filtered = data_filtered.sort_values(["RATIO_(prev_two_col)", "PATIENT_INTERFACE_COUNT"], ascending=[False, False])

    return data_filtered


In [5]:
brca_preliminary_data = pd.read_excel(BRCA_PRELIMINARY_DATA_PATH)
coad_preliminary_data = pd.read_excel(COAD_PRELIMINARY_DATA_PATH)
esca_preliminary_data = pd.read_excel(ESCA_PRELIMINARY_DATA_PATH)
gbm_preliminary_data = pd.read_excel(GBM_PRELIMINARY_DATA_PATH)
hnsc_preliminary_data = pd.read_excel(HNSC_PRELIMINARY_DATA_PATH)
ov_preliminary_data = pd.read_excel(OV_PRELIMINARY_DATA_PATH)

In [6]:
brca_preliminary_data_filtered = filterer(brca_preliminary_data, "BRCA")
coad_preliminary_data_filtered = filterer(coad_preliminary_data, "COAD")
esca_preliminary_data_filtered = filterer(esca_preliminary_data, "ESCA")
gbm_preliminary_data_filtered = filterer(gbm_preliminary_data, "GBM")
hnsc_preliminary_data_filtered = filterer(hnsc_preliminary_data, "HNSC")
ov_preliminary_data_filtered = filterer(ov_preliminary_data, "OV")

In [7]:
brca_preliminary_data_filtered_updated = MostFrequentlyDisruptedPartnersAdder(
    tcga="BRCA",
    preliminary_data=brca_preliminary_data_filtered,
    interactions_summary_folder_path=INTERACTIONS_SUMMARY_TABLE_FOLDER_PATH,
).get_most_frequently_disrupted_partners_added_data()

coad_preliminary_data_filtered_updated = MostFrequentlyDisruptedPartnersAdder(
    tcga="COAD",
    preliminary_data=coad_preliminary_data_filtered,
    interactions_summary_folder_path=INTERACTIONS_SUMMARY_TABLE_FOLDER_PATH,
).get_most_frequently_disrupted_partners_added_data()

esca_preliminary_data_filtered_updated = MostFrequentlyDisruptedPartnersAdder(
    tcga="ESCA",
    preliminary_data=esca_preliminary_data_filtered,
    interactions_summary_folder_path=INTERACTIONS_SUMMARY_TABLE_FOLDER_PATH,
).get_most_frequently_disrupted_partners_added_data()

gbm_preliminary_data_filtered_updated = MostFrequentlyDisruptedPartnersAdder(
    tcga="GBM",
    preliminary_data=gbm_preliminary_data_filtered,
    interactions_summary_folder_path=INTERACTIONS_SUMMARY_TABLE_FOLDER_PATH,
).get_most_frequently_disrupted_partners_added_data()

hnsc_preliminary_data_filtered_updated = MostFrequentlyDisruptedPartnersAdder(
    tcga="HNSC",
    preliminary_data=hnsc_preliminary_data_filtered,
    interactions_summary_folder_path=INTERACTIONS_SUMMARY_TABLE_FOLDER_PATH,
).get_most_frequently_disrupted_partners_added_data()

ov_preliminary_data_filtered_updated = MostFrequentlyDisruptedPartnersAdder(
    tcga="OV",
    preliminary_data=ov_preliminary_data_filtered,
    interactions_summary_folder_path=INTERACTIONS_SUMMARY_TABLE_FOLDER_PATH,
).get_most_frequently_disrupted_partners_added_data()

In [8]:
brca_preliminary_data_filtered

,TCGA,PROTEIN,GENE,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
2465,BRCA,P42336,PIK3CA,35,124,120,0.97,6,+
1204,BRCA,P04637,TP53,141,55,52,0.95,17,+
3126,BRCA,P68431,H3C1,1,13,12,0.92,11,-
3091,BRCA,P62805,H4C1,6,11,10,0.91,16,-
2534,BRCA,P45985,MAP2K4,3,8,7,0.88,7,+
1201,BRCA,P04626,ERBB2,12,14,12,0.86,16,+
1447,BRCA,P0CG48,UBC,0,7,6,0.86,7,-


In [9]:
coad_preliminary_data_filtered

,TCGA,PROTEIN,GENE,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
1489,COAD,P04637,TP53,97,49,49,1.00,19,+
2228,COAD,P19784,CSNK2A2,2,9,9,1.00,10,-
4709,COAD,Q15365,PCBP1,4,9,9,1.00,5,+
212,COAD,O00238,BMPR1B,2,6,6,1.00,16,-
2110,COAD,P16885,PLCG2,4,6,6,1.00,14,-
3870,COAD,P84022,SMAD3,5,6,6,1.00,5,+
7578,COAD,Q96E17,RAB3C,1,6,6,1.00,4,-
2968,COAD,P41743,PRKCI,2,5,5,1.00,13,-
3453,COAD,P53350,PLK1,5,5,5,1.00,10,-
3538,COAD,P55072,VCP,3,5,5,1.00,11,-


In [10]:
esca_preliminary_data_filtered

,TCGA,PROTEIN,GENE,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
627,ESCA,P04637,TP53,56,35,34,0.97,17,+
1571,ESCA,P62805,H4C1,2,7,6,0.86,11,-


In [11]:
gbm_preliminary_data_filtered

,TCGA,PROTEIN,GENE,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
1050,GBM,P04637,TP53,52,35,35,1.0,19,+
1449,GBM,P15056,BRAF,1,5,4,0.8,5,+
2925,GBM,Q05655,PRKCD,2,5,4,0.8,10,-


In [12]:
hnsc_preliminary_data_filtered

,TCGA,PROTEIN,GENE,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
1012,HNSC,P01112,HRAS,1,26,26,1.00,26,+
2792,HNSC,P63000,RAC1,0,11,11,1.00,8,+
1499,HNSC,P15056,BRAF,0,6,6,1.00,6,+
2381,HNSC,P49336,CDK8,2,6,6,1.00,3,-
3410,HNSC,Q14790,CASP8,13,5,5,1.00,5,+
1099,HNSC,P04637,TP53,134,59,58,0.98,16,+
5467,HNSC,Q969H0,FBXW7,2,18,17,0.94,3,+
2220,HNSC,P42336,PIK3CA,13,14,13,0.93,5,+
2236,HNSC,P42771,CDKN2A,1,19,17,0.89,23,+
1754,HNSC,P22607,FGFR3,0,6,5,0.83,20,+


In [13]:
ov_preliminary_data_filtered

,TCGA,PROTEIN,GENE,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
927,OV,P04637,TP53,147,80,80,1.0,19,+
782,OV,O95835,LATS1,1,5,4,0.8,7,+


In [14]:
brca_preliminary_data_filtered_updated

,TCGA,PROTEIN,GENE,MOST_FREQUENTLY_DISRUPTED_PARTNER,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
2465,BRCA,P42336,PIK3CA,"PIK3R1, PIK3R2 (120)",35,124,120,0.97,6,+
1204,BRCA,P04637,TP53,"TP53BP2, PPP1R13B (41)",141,55,52,0.95,17,+
3126,BRCA,P68431,H3C1,DAXX (6),1,13,12,0.92,11,-
3091,BRCA,P62805,H4C1,H3C1 (8),6,11,10,0.91,16,-
2534,BRCA,P45985,MAP2K4,"LRRK2, MAP2K6 (6)",3,8,7,0.88,7,+
1201,BRCA,P04626,ERBB2,SRPK1 (7),12,14,12,0.86,16,+
1447,BRCA,P0CG48,UBC,UBC1 (4),0,7,6,0.86,7,-


In [15]:
concated_preliminary_filtered_data_updated = pd.concat(
    [
        brca_preliminary_data_filtered_updated,
        coad_preliminary_data_filtered_updated,
        esca_preliminary_data_filtered_updated,
        gbm_preliminary_data_filtered_updated,
        hnsc_preliminary_data_filtered_updated,
        ov_preliminary_data_filtered_updated,
    ], ignore_index=True
)

In [16]:
concated_preliminary_filtered_data_updated

,TCGA,PROTEIN,GENE,MOST_FREQUENTLY_DISRUPTED_PARTNER,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
0,BRCA,P42336,PIK3CA,"PIK3R1, PIK3R2 (120)",35,124,120,0.97,6,+
1,BRCA,P04637,TP53,"TP53BP2, PPP1R13B (41)",141,55,52,0.95,17,+
2,BRCA,P68431,H3C1,DAXX (6),1,13,12,0.92,11,-
3,BRCA,P62805,H4C1,H3C1 (8),6,11,10,0.91,16,-
4,BRCA,P45985,MAP2K4,"LRRK2, MAP2K6 (6)",3,8,7,0.88,7,+
5,BRCA,P04626,ERBB2,SRPK1 (7),12,14,12,0.86,16,+
6,BRCA,P0CG48,UBC,UBC1 (4),0,7,6,0.86,7,-
7,COAD,P04637,TP53,"TP53BP2, PPP1R13B (41)",97,49,49,1.00,19,+
8,COAD,P19784,CSNK2A2,PAK1 (4),2,9,9,1.00,10,-
9,COAD,Q15365,PCBP1,"HNRNPK, PCBP2, PCBP4 (9)",4,9,9,1.00,5,+


In [17]:
# very few `nan`s so I can fix it by hand.
concated_preliminary_filtered_data_updated[
    concated_preliminary_filtered_data_updated["MOST_FREQUENTLY_DISRUPTED_PARTNER"].str.contains("nan")
]

,TCGA,PROTEIN,GENE,MOST_FREQUENTLY_DISRUPTED_PARTNER,PATIENT_CORE_COUNT,PATIENT_INTERFACE_COUNT,NUM_INTERFACE_PATIENTS_DISRUPTIVE_INTERACTOR,RATIO_(prev_two_col),NUM_UNIQUE_INTERACTORS,CGC_STATUS
39,ESCA,P62805,H4C1,"H3C1, DAXX, nan (3)",2,7,6,0.86,11,-
47,HNSC,Q14790,CASP8,"CASP7, nan (3)",13,5,5,1.00,5,+


[14:29, 03/03/2022] Hilal Kazan (ABU CS Prof. ): COADS Q13748 için TUBA3C gen id'si

In [23]:
concated_preliminary_filtered_data_updated.to_clipboard(index=False, float_format='%.2f')

- - -